In [305]:
import requests
from bs4 import BeautifulSoup
# import lxml.html as lh
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import numpy as np
import re
import time
import json
from tqdm import tqdm
from sqlalchemy import create_engine

from bokeh.models.glyphs import Arc
from math import pi
from bokeh.plotting import figure, output_notebook, show

from bokeh.palettes import Reds6,Blues8,RdYlBu,Reds,Inferno256

In [320]:
from bokeh.io import reset_output
import numpy as np

from bokeh.io import output_file, show, reset_output
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.models import ColorBar,LinearColorMapper,LogColorMapper,ContinuousColorMapper
from bokeh.util.hex import hexbin
from bokeh.palettes import RdBu,Oranges,Turbo256,RdPu
from bokeh.models import HoverTool

In [2]:
from Scraper import get_schedule

# Drawing

In [6]:
def get_games_events(teamid=None,scheduleid_list=None):
    """
    从database中，根据teamid或scheduleid，获取比赛的所有events，包括对手的events。
    注意，即使值提供teamid参数，也会获取对手的events。
    
    """
    assert (not (teamid is None)&(scheduleid_list is None)),'至少需要teamid或scheduleid_list一个参数'
    user_name = 'master'
    passcode = 'Pw#cbashuju0131'
    endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
    database = 'CBA_Data'
    #     database = 'CBA_Staging'
    engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')
    connection= engine.connect()
    if scheduleid_list:
        assert type(scheduleid_list)==list,'scheduleid_list只接受list'
        sql_str = f"""
        SELECT *
        FROM CBA_Data.Playbyplay plays1
        WHERE plays1.scheduleid IN ('{"','".join(str(i) for i in scheduleid_list)}')
        """
    else:
        sql_str = f"""
        SELECT *
        FROM CBA_Data.Playbyplay plays1
        WHERE plays1.scheduleid IN (
            SELECT DISTINCT plays2.scheduleid
            FROM CBA_Data.Playbyplay plays2
            WHERE plays2.teamid = {teamid}
        )
        """
    team_events = pd.read_sql(sql_str, connection)
    # plays.to_sql(name='Playbyplay',con=connection,index=False,if_exists='replace')
    connection.close()
    return team_events

In [64]:
def get_shotevents(games_events,teamid=None,player_name=None):
    """
    此函数能够根据输入的playbyplay数据，筛选出投篮事件。
    若指定某队teamid，则可以筛选特定某队的投篮事件。
    若指定某队teamid和球员名字，则可以筛选特定某队员的投篮事件。
    """
    games_shotevents = games_events.loc[pd.notna(games_events['shotevent']),].copy()

    games_shotevents['shotcoordx'] = games_shotevents['shotcoordx'].astype('float')
    games_shotevents['shotcoordy'] = games_shotevents['shotcoordy'].astype('float')
    
    if teamid:
        games_shotevents = games_shotevents.loc[games_shotevents['teamid']==teamid]
        if player_name:
            games_shotevents = games_shotevents.loc[(games_shotevents['event'].str.contains('player_name'))&(-games_shotevents['event'].str.contains(player_name+'抢到'))&(-(games_shotevents['event'].str.contains(player_name+'的助攻'))),:]
            
    
    
    return games_shotevents

### 球队投篮分布

In [28]:
import math
from math import pi

In [24]:
def meter_to_coord(meter):
    return meter*49/15

In [25]:
def angle(vector1, vector2):
    x1, y1 = vector1
    x2, y2 = vector2
    inner_product = x1*x2 + y1*y2
    len1 = math.hypot(x1, y1)
    len2 = math.hypot(x2, y2)
    return math.acos(inner_product/(len1*len2))

In [30]:
def draw_court(p):
    endline_y = -7.5
    halfcourtline_y=45.73
    sideline_left = 24.5
    rim_to_baseline=7.5
    rim_x=0
    rim_y=endline_y+meter_to_coord(1.2)
    threepointline_radius = meter_to_coord(6.75)
    nochargeline_rarius = meter_to_coord(1.25)
    centercircle_radius = meter_to_coord(1.83)
    rim_radius = meter_to_coord(0.23)
    backboard_length = meter_to_coord(1.83)
    restrictedarea_length = meter_to_coord(5.8)
    restrictedarea_width = meter_to_coord(4.9)
    ftline_length = meter_to_coord(3.6)


    goal_color = Blues8[1]
    miss_color = Reds6[0]
    linewidth = 3
    size = 9
    alpha = 0.6
    
    # 3p line
    p.arc(x=rim_x, y=rim_y, radius=threepointline_radius, start_angle=0, end_angle=pi,line_color='black', line_width=linewidth)

    # 左侧三分直线
    p.line(x=[-threepointline_radius,-threepointline_radius],y=[endline_y,rim_y],line_color='black', line_width=linewidth-0.1)

    # 右侧三分直线
    p.line(x=[threepointline_radius,threepointline_radius],y=[endline_y,rim_y],line_color='black', line_width=linewidth-0.1)

    # 合理冲撞区
    p.arc(x=rim_x, y=rim_y, radius=nochargeline_rarius, start_angle=0, end_angle=pi,line_color='black', line_width=linewidth)

    # 中圈
    p.arc(x=rim_x, y=halfcourtline_y, radius=centercircle_radius, start_angle=pi, end_angle=0,line_color='black', line_width=linewidth)

    # 篮筐
    p.circle(x=0,y=rim_y,radius = rim_radius,color='red',fill_color=None,line_width=linewidth)


    # 篮板
    p.line(x=[-backboard_length/2,backboard_length/2],
           y=[rim_y-rim_radius-0.3,rim_y-rim_radius-0.3],
           line_color='black', line_width=linewidth)

    #场地边线
    p.quad(-sideline_left,sideline_left,halfcourtline_y,endline_y,fill_color=None,line_color='black',
          line_width=linewidth)

    #限制区
    p.quad(-restrictedarea_width/2,restrictedarea_width/2,endline_y+restrictedarea_length,endline_y,fill_color=None,line_color='black',
          line_width=linewidth)

    #罚球区
    p.arc(x=rim_x, y=endline_y+restrictedarea_length, radius=ftline_length/2, start_angle=0, end_angle=pi,line_color='black', line_width=linewidth)


In [340]:
def plot_shot_dist_dot(df_shotevents,plot_title='投篮分布图'):

    output_notebook()

    # create a new plot
    p = figure(
       title=plot_title,tools="pan,box_zoom,wheel_zoom,save,reset,hover",
        x_range=[-sideline_left,sideline_left],y_range = [endline_y,halfcourtline_y],x_axis_label='x',y_axis_label='y'
    )


    p.axis.visible = False
    p.grid.visible = False

    # misses
    p.cross(df_shotevents.loc[df_shotevents['eventtypeid']=='4','shotcoordx'],df_shotevents.loc[df_shotevents['eventtypeid']=='4','shotcoordy'],
            angle=90,color=miss_color, size=size,alpha=alpha,legend_label='未命中')

    # goals
    p.circle(df_shotevents.loc[df_shotevents['eventtypeid']=='3','shotcoordx'], df_shotevents.loc[df_shotevents['eventtypeid']=='3','shotcoordy'], 
             fill_color=goal_color,color=goal_color, size=size,alpha=alpha,legend_label='命中')

    draw_court(p)

    # show the results
    show(p)

    #width: 15m, 50
    #length: 28m，

def plot_shot_dist_hex(df_shotevents,plot_title='投篮分布图',palette = inferno(256),
                       hex_size=2,dot_size=5,thres_low=10,thres_high=300):
    reset_output()
    output_notebook()

    x = df_shotevents['shotcoordx'].values
    y = df_shotevents['shotcoordy'].values
    
    color_grade = palette
    
    bins = hexbin(x, y, hex_size)
    
    p = figure(
       title=plot_title,tools="save,reset",
        x_range=[-sideline_left,sideline_left],y_range = [endline_y,halfcourtline_y],x_axis_label='x',y_axis_label='y',
        match_aspect=True, background_fill_color=None
    )
   
    p.grid.visible = False
    p.axis.visible = False

    p.hex_tile(q="q", r="r", size=hex_size, line_color=None, source=bins, 
               fill_color=linear_cmap('counts', color_grade, low=thres_low, 
                                      high=max(bins.counts),low_color='white')
               ,alpha=0.8)
    
#     p.dot(df_shotevents['shotcoordx'], df_shotevents['shotcoordy'], 
#          fill_color='red',color='red', size=dot_size,alpha=1,legend_label='出手')
    
    color_mapper = LinearColorMapper(palette=palette, low=thres_low, high=thres_high,low_color='white')
    color_bar = ColorBar(title='出手频率',color_mapper=color_mapper, label_standoff=12, border_line_color=None, location=(0,0))
    p.add_layout(color_bar, 'right')
    
    draw_court(p)
    show(p)

In [336]:
mycolor = list(Inferno256)[75:150]

In [326]:
# mycolor.reverse()

In [5]:
liaoning = '29129'
guangdong = '29124'

In [341]:

teamid=liaoning
games_events = get_games_events(teamid)
liaoning_shotevents = get_shotevents(games_events,teamid=liaoning)

plot_shot_dist_dot(liaoning_shotevents)

plot_shot_dist_hex(liaoning_shotevents,plot_title='辽宁投篮分布图',palette = RdBu[11],
                       hex_size=2,dot_size=5,thres_low=40,thres_high=300)

Loading BokehJS ...

Loading BokehJS ...

In [342]:
teamid=guangdong
games_events = get_games_events(teamid)
guangdong_shotevents = get_shotevents(games_events,teamid=guangdong)

plot_shot_dist_dot(guangdong_shotevents)

plot_shot_dist_hex(guangdong_shotevents,plot_title='广东投篮分布图',palette = RdBu[11],
                       hex_size=2,dot_size=5,thres_low=40,thres_high=300)

Loading BokehJS ...

Loading BokehJS ...

### 周琦vs易建联

In [227]:
zq_shots = xj_shots.loc[(xj_shots['event'].str.contains('周琦'))&(-xj_shots['event'].str.contains('周琦抢到'))&(-(xj_shots['event'].str.contains('周琦的助攻'))),:].copy()


In [226]:
yjl_shots = gd_shots.loc[(gd_shots['event'].str.contains('易建联'))&(-gd_shots['event'].str.contains('易建联抢到'))&(-(gd_shots['event'].str.contains('易建联的助攻'))),:].copy()


In [241]:
print(f'易建联共{296+94}投{168+34}中，总命中率{(168+34)/(296+94)*100:.2f}%，3分投篮占比{94/(296+94)*100:.2f}%')

易建联共390投202中，总命中率51.79%，3分投篮占比24.10%


In [240]:
print(f'周琦共{243+103}投{33+157}中，总命中率{(33+157)/(243+103)*100:.2f}%，3分投篮占比{103/(243+103)*100:.2f}%')

周琦共346投190中，总命中率54.91%，3分投篮占比29.77%


In [379]:
from bokeh.models.glyphs import Arc
from math import pi

# prepare some data

# output to static HTML file
output_notebook()

# create a new plot
p = figure(
   title="周琦19-20赛季投篮分布--共346投190中，总命中率54.91%，3分投篮占比29.77%",tools="pan,box_zoom,wheel_zoom,save,reset,hover",
    x_range=[-24.5,24.5],y_range = [-7.5,45.73],x_axis_label='x',y_axis_label='y'
)

p.axis.visible = False

# misses
p.cross(zq_shots.loc[zq_shots['eventtypeid']=='4','shotcoordx'],zq_shots.loc[zq_shots['eventtypeid']=='4','shotcoordy'],
        angle=90,color=miss_color, size=10,alpha=alpha,legend_label='未命中')

# goals
p.circle(zq_shots.loc[zq_shots['eventtypeid']=='3','shotcoordx'], zq_shots.loc[zq_shots['eventtypeid']=='3','shotcoordy'], 
         fill_color=goal_color,color=goal_color, size=10,alpha=alpha,legend_label='命中')

draw_court(p)

# show the results
show(p)

#width: 15m, 50
#length: 28m，


Loading BokehJS ...

In [380]:
# prepare some data

# output to static HTML file
output_notebook()

# create a new plot
p = figure(
   title="易建联19-20赛季投篮分布--共390投202中，总命中率51.79%，3分投篮占比24.10%",tools="pan,box_zoom,wheel_zoom,save,reset,hover",
    x_range=[-24.5,24.5],y_range = [-7.5,45.73],x_axis_label='xxx',y_axis_label='yyy'
)
p.axis.visible = False

# misses
p.cross(yjl_shots.loc[yjl_shots['eventtypeid']=='4','shotcoordx'],yjl_shots.loc[yjl_shots['eventtypeid']=='4','shotcoordy'],
        angle=90,color=miss_color, size=10,alpha=alpha,legend_label='未命中')

# goals
p.circle(yjl_shots.loc[yjl_shots['eventtypeid']=='3','shotcoordx'], yjl_shots.loc[yjl_shots['eventtypeid']=='3','shotcoordy'], 
         fill_color=goal_color,color=goal_color, size=10,alpha=alpha,legend_label='命中')

draw_court(p)

# show the results
show(p)

#width: 15m, 50
#length: 28m，


Loading BokehJS ...